In [1]:
# Experiment: 2021/03/14
# Usage: Dp analysis of mnist neural network.
# Detail: Check the notion notes in HE-DP project.

from dpsgd_keras_slow import *

dpsgd = False # add dp noise or not 
learning_rate = 0.1
noise_multiplier = 1
l2_norm_clip = 3
batch_size = 1024
epochs = 40
microbatch_size = 16
num_parameters = 0
privacy_budget = []
delta = 1e-5  # it is recommended to use delta~=1/dataset_size
model_dir = None

In [2]:
tf.__version__

'2.4.1'

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7,8'

In [4]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [7]:
train, test = tf.keras.datasets.cifar10.load_data()

In [8]:
def load_mnist():
  """Loads MNIST and preprocesses to combine training and validation data."""
  train, test = tf.keras.datasets.cifar10.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  train_data = train_data.reshape((train_data.shape[0], 32, 32, 3))
  test_data = test_data.reshape((test_data.shape[0], 32, 32, 3))

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  assert train_data.min() == 0.
  assert train_data.max() == 1.
  assert test_data.min() == 0.
  assert test_data.max() == 1.

  return train_data, train_labels, test_data, test_labels

In [9]:
# Perturnbing the input dataset, and collect the accuracy-step
class GaussianNoiseLayer(tf.keras.layers.Layer):
    def __init__(self, l2_norm_clip, noise_multiplier, *args, **kwargs):
        super(GaussianNoiseLayer, self).__init__(*args, **kwargs)
        self._l2_norm_clip = l2_norm_clip
        self._noise_multiplier = noise_multiplier

    def build(self, input_shape):
        pass
    
    def call(self, x):
        # Clip gradients to given l2_norm_clip.
        def clip_features(x):
            return tf.clip_by_global_norm([x], self._l2_norm_clip)[0][0]

        clipped_features = tf.map_fn(clip_features, x)
        
        # Add noise to summed gradients.
        noise_stddev = self._l2_norm_clip * self._noise_multiplier
        noise = tf.random.normal(tf.shape(input=clipped_features), stddev=noise_stddev)
        return clipped_features + noise

In [10]:
#tf.clip_by_global_norm([tf.Variable([[2,3],[4,5]],shape=[2,2],dtype=tf.float32)], 4.4)[0][0]

In [11]:
# Models 
def build_models(noise_layer_name):
    if noise_layer_name =="cifar10+untied_bias+noise_input":
        model = tf.keras.Sequential([
            GaussianNoiseLayer(l2_norm_clip, noise_multiplier, input_shape=(32, 32, 3)),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
    elif noise_layer_name == "cifar10+tied_bias+noise_input":
        model = tf.keras.Sequential([
            GaussianNoiseLayer(l2_norm_clip, noise_multiplier, input_shape=(32, 32, 3)),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
    elif noise_layer_name == "cifar10+untied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
    elif noise_layer_name == "cifar10+tied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])
    elif noise_layer_name == "cifar10":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False),
            BiasLayer()
        ])  
    elif noise_layer_name == "sphinx+cifar10+untied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered2"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered3"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered4"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered5"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered6"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered7"),
            BiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
            BiasLayer()
        ])
    elif noise_layer_name == "sphinx+cifar10+tied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered2"),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered3"),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered4"),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered5"),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered6"),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered7"),
            TiedBiasLayer(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
            BiasLayer()
        ])        
    elif noise_layer_name == "ALLnoise+cifar10+untied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
            BiasLayer(name="Considered9"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered2"),
            BiasLayer(name="Considered10"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered3"),
            BiasLayer(name="Considered11"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered4"),
            BiasLayer(name="Considered12"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered5"),
            BiasLayer(name="Considered13"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered6"),
            BiasLayer(name="Considered14"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered7"),
            BiasLayer(name="Considered15"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
            BiasLayer(name="Considered16")
        ])
    elif noise_layer_name == "ALLnoise+cifar10+tied_bias":
        model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None,
                                 input_shape=(32, 32, 3), use_bias=False, name="Considered1"),
            TiedBiasLayer(name="Considered9"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered2"),
            TiedBiasLayer(name="Considered10"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered3"),
            TiedBiasLayer(name="Considered11"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered4"),
            TiedBiasLayer(name="Considered12"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2), 2),
            tf.keras.layers.Conv2D(64, 3,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered5"),
            TiedBiasLayer(name="Considered13"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(64, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered6"),
            TiedBiasLayer(name="Considered14"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(16, 1,
                                 strides=1,
                                 padding='same',
                                 activation=None, use_bias=False, name="Considered7"),
            TiedBiasLayer(name="Considered15"),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation=None, use_bias=False, name="Considered8"),
            BiasLayer(name="Considered16")
        ])
    else:
        model = None
    return model

In [12]:
build_models("cifar10+untied_bias+noise_input").summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_layer (Gaussi (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1728      
_________________________________________________________________
bias_layer (BiasLayer)       (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu (ReLU)                 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36864     
_________________________________________________________________
bias_layer_1 (BiasLayer)     (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 64)        0

In [13]:
build_models("cifar10+untied_bias").summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 64)        1728      
_________________________________________________________________
bias_layer_8 (BiasLayer)     (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu_7 (ReLU)               (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        36864     
_________________________________________________________________
bias_layer_9 (BiasLayer)     (None, 32, 32, 64)        65536     
_________________________________________________________________
re_lu_8 (ReLU)               (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)       

In [14]:
build_models("cifar10+tied_bias").summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 64)        1728      
_________________________________________________________________
tied_bias_layer (TiedBiasLay (None, 32, 32, 64)        64        
_________________________________________________________________
re_lu_14 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 64)        36864     
_________________________________________________________________
tied_bias_layer_1 (TiedBiasL (None, 32, 32, 64)        64        
_________________________________________________________________
re_lu_15 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)       

In [15]:
build_models("cifar10").summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 64)        1728      
_________________________________________________________________
re_lu_21 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 64)        36864     
_________________________________________________________________
re_lu_22 (ReLU)              (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 64)        36864     
_________________________________________________________________
re_lu_23 (ReLU)              (None, 16, 16, 64)       

In [16]:
#def main_simple(unused_argv):
logging.set_verbosity(logging.INFO)
if dpsgd and batch_size % microbatch_size != 0:
    raise ValueError('Number of microbatches should divide evenly batch_size')

# Load training and test data.
train_data, train_labels, test_data, test_labels = load_mnist()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=(epochs * train_data.shape[0]) // microbatch_size // 1000,
    decay_rate=0.998)

In [17]:
# check the training accuracy
dpsgd = True
parameter_list = {}
accuracies = {}
#for model_type in ["mnist", "mnist+tied_bias", "mnist+untied_bias"]:
for model_type in ["sphinx+cifar10+tied_bias", "ALLnoise+cifar10+tied_bias","cifar10+tied_bias", "cifar10+tied_bias+noise_input"]:
    file_name = model_type + "_accuracy"
    if model_type == "cifar10+tied_bias+noise_input":
        dpsgd = False
    else:
        dpsgd = True
    model = build_models(model_type)
    if dpsgd:
        optimizer = FixedDPKerasSGDOptimizer(
            batch_size = batch_size,
            num_parameters = num_parameters,
            l2_norm_clip=l2_norm_clip,
            noise_multiplier=noise_multiplier,
            var_list = model.trainable_variables,
            #num_microbatches=batch_size//microbatch_size,
            microbatch_size = microbatch_size,
            learning_rate=learning_rate)
        # Compute vector of per-example loss rather than its mean over a minibatch.
        loss = tf.keras.losses.CategoricalCrossentropy(
            from_logits=True, reduction=tf.losses.Reduction.NONE)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile model with Keras
    checkpoint_filepath = "./dp_data/"+file_name
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True)
    #early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    ##    monitor='val_loss', min_delta=0, patience=5, verbose=0,
    #    mode='auto', baseline=None, restore_best_weights=True
    #)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    # Train model with Keras
    if dpsgd:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=microbatch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    else:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=batch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    
    #evaluated_result = model.evaluate(
    #    x=test_data, y=test_labels, batch_size=None, verbose=1, sample_weight=None, steps=None,
    #    callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False,
    #    return_dict=True)
    if model_type not in accuracies.keys():
        accuracies[model_type] = []
    accuracies[model_type].append(history)
    #accuracies[model_type].append(evaluated_result["accuracy"])

# Compute the privacy budget expended.
#if dpsgd:
#    eps = compute_epsilon(epochs * 60000 // batch_size)
#    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
#else:
#    print('Trained with vanilla non-private SGD optimizer')

Epoch 1/40
compute the gradient
[True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False]
clipped_gradients: [<tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack/TensorListStack:0' shape=(16, 3, 3, 3, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_1/TensorListStack:0' shape=(16, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_2/TensorListStack:0' shape=(16, 3, 3, 64, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_3/TensorListStack:0' shape=(16, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_4/TensorListStack:0' shape=(16, 3, 3, 64, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_5/TensorListStack:0' shape=(16, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_6/TensorListStack:0' shape=(16, 3, 3, 64, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_7/TensorListStack:0' shape=(16, 64) dtype=float32>

Epoch 17/40
3125/3125 [==============================] - 116s 37ms/step - loss: 1.8662 - accuracy: 0.3641 - val_loss: 1.9180 - val_accuracy: 0.3576
Epoch 18/40
3125/3125 [==============================] - 116s 37ms/step - loss: 1.8670 - accuracy: 0.3663 - val_loss: 1.8456 - val_accuracy: 0.3734
Epoch 19/40
3125/3125 [==============================] - 116s 37ms/step - loss: 1.8797 - accuracy: 0.3657 - val_loss: 1.9018 - val_accuracy: 0.3637
Epoch 20/40
3125/3125 [==============================] - 116s 37ms/step - loss: 1.8790 - accuracy: 0.3704 - val_loss: 1.8077 - val_accuracy: 0.3826
Epoch 21/40
3125/3125 [==============================] - 116s 37ms/step - loss: 1.8411 - accuracy: 0.3824 - val_loss: 1.9186 - val_accuracy: 0.3626
Epoch 22/40
3125/3125 [==============================] - 115s 37ms/step - loss: 1.8553 - accuracy: 0.3798 - val_loss: 1.9105 - val_accuracy: 0.3763
Epoch 23/40
3125/3125 [==============================] - 115s 37ms/step - loss: 1.8752 - accuracy: 0.3788 - val_

3125/3125 [==============================] - 122s 38ms/step - loss: 2.2929 - accuracy: 0.1433 - val_loss: 2.2451 - val_accuracy: 0.1694
Epoch 2/40
3125/3125 [==============================] - 121s 39ms/step - loss: 2.2207 - accuracy: 0.1859 - val_loss: 2.1363 - val_accuracy: 0.2304
Epoch 3/40
3125/3125 [==============================] - 121s 39ms/step - loss: 2.1204 - accuracy: 0.2200 - val_loss: 2.0943 - val_accuracy: 0.2360
Epoch 4/40
3125/3125 [==============================] - 122s 39ms/step - loss: 2.0882 - accuracy: 0.2432 - val_loss: 2.2392 - val_accuracy: 0.2504
Epoch 5/40
3125/3125 [==============================] - 121s 39ms/step - loss: 2.0855 - accuracy: 0.2558 - val_loss: 2.0812 - val_accuracy: 0.2602
Epoch 6/40
3125/3125 [==============================] - 121s 39ms/step - loss: 2.0484 - accuracy: 0.2737 - val_loss: 1.9957 - val_accuracy: 0.2986
Epoch 7/40
3125/3125 [==============================] - 120s 38ms/step - loss: 2.0372 - accuracy: 0.2854 - val_loss: 1.9689 - val

compute the gradient
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
clipped_gradients: [<tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack/TensorListStack:0' shape=(16, 3, 3, 3, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_1/TensorListStack:0' shape=(16, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_2/TensorListStack:0' shape=(16, 3, 3, 64, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_3/TensorListStack:0' shape=(16, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_4/TensorListStack:0' shape=(16, 3, 3, 64, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_5/TensorListStack:0' shape=(16, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_6/TensorListStack:0' shape=(16, 3, 3, 64, 64) dtype=float32>, <tf.Tensor 'SGD/gradients/map/TensorArrayV2Stack_7/TensorListStack:0' shape=(16, 64) dtype=float32>, <

Epoch 37/40
3125/3125 [==============================] - 85s 27ms/step - loss: 1.0376 - accuracy: 0.6355 - val_loss: 1.0804 - val_accuracy: 0.6199
Epoch 38/40
3125/3125 [==============================] - 85s 27ms/step - loss: 1.0092 - accuracy: 0.6480 - val_loss: 1.0867 - val_accuracy: 0.6159
Epoch 39/40
3125/3125 [==============================] - 86s 28ms/step - loss: 0.9975 - accuracy: 0.6520 - val_loss: 1.1085 - val_accuracy: 0.6141
Epoch 40/40
3125/3125 [==============================] - 87s 28ms/step - loss: 0.9568 - accuracy: 0.6614 - val_loss: 1.0238 - val_accuracy: 0.6395
Epoch 1/40
49/49 [==============================] - 306s 4s/step - loss: 2.3112 - accuracy: 0.0991 - val_loss: 2.3029 - val_accuracy: 0.0958
Epoch 2/40
49/49 [==============================] - 20s 400ms/step - loss: 2.3032 - accuracy: 0.0975 - val_loss: 2.3029 - val_accuracy: 0.0987
Epoch 3/40
49/49 [==============================] - 20s 402ms/step - loss: 2.3029 - accuracy: 0.0994 - val_loss: 2.3028 - val_ac

In [18]:
accuracies

{'sphinx+cifar10+tied_bias': [<tensorflow.python.keras.callbacks.History at 0x7f51681eff10>],
 'ALLnoise+cifar10+tied_bias': [<tensorflow.python.keras.callbacks.History at 0x7f51604d41d0>],
 'cifar10+tied_bias': [<tensorflow.python.keras.callbacks.History at 0x7f5468946a50>],
 'cifar10+tied_bias+noise_input': [<tensorflow.python.keras.callbacks.History at 0x7f5468a4e710>]}

In [19]:

processed_accuracies = dict()
for key in accuracies.keys():   
    processed_accuracies[key] = accuracies[key][0].history
    
import pickle
file = open("dp_data/results/cifar10_accuracies_%.1f_%.1f_%.1f_%d"%(learning_rate, noise_multiplier, l2_norm_clip, batch_size),"wb")
pickle.dump(processed_accuracies, file)
file.close()

In [20]:
assert(False)

AssertionError: 

In [ ]:
processed_accuracies

## Perturbed Label : skip


In [ ]:
def clip_noise(x, clip, noise_multiplier):
    def clip_features(x):
        return tf.clip_by_global_norm([x], clip)[0][0]
    clipped_features = tf.map_fn(clip_features, x)
    # Add noise to summed gradients.
    noise_stddev = clip * noise_multiplier
    noise = tf.random.normal(tf.shape(input=clipped_features), stddev=noise_stddev)
    return clipped_features + noise

noisy_train_data = tf.map_fn(lambda x: clip_noise(x, l2_norm_clip, noise_multiplier), train_data)

In [ ]:
accuracies

In [ ]:
processed_accuracies = dict()
for key in accuracies.keys():   
    processed_accuracies[key] = accuracies[key][0].history

In [ ]:
processed_accuracies

In [ ]:
import matplotlib.pyplot as plt
plt.plot(processed_accuracies['sphinx+mnist+untied_bias']['accuracy'])
plt.plot(processed_accuracies['sphinx+mnist+untied_bias']['val_accuracy'])
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import pickle
file = open("dp_data/results/accuracies_%.1f_%.1f_%.1f_%d"%(learning_rate, noise_multiplier, l2_norm_clip, batch_size),"wb")
pickle.dump(processed_accuracies, file)
file.close()

In [ ]:
assert(False)

## get the step-epsilon curve

In [ ]:
import numpy as np
import pickle 

try:
    with open("dp_data/noise_epsilon_step") as f:
        epsilons = pickle.load(f)
except:
    training_data_size = 60000
    max_step = 25000
    epsilons = [[] for i in range(10)]
    for noise_multiplier in range(0, 10, 1):
        print("noise_multiplier:", noise_multiplier)
        for step in range(1, max_step, 10):
            epsilons[noise_multiplier].append(compute_epsilon(step, batch_size/training_data_size, noise_multiplier))
    file = open("dp_data/noise_epsilon_step_%d"%batch_size,"wb")
    pickle.dump(epsilons, file)
    file.close()

## Tied Bias

In [ ]:
# For tied bias
#for model_type in ["mnist", "mnist+tied_bias", "mnist+untied_bias"]:
for model_type in ["sphinx+mnist+tied_bias", "ALLnoise+mnist+tied_bias","mnist+tied_bias"]:
    file_name = model_type + "_accuracy"
    if model_type == "mnist+tied_bias+noise_input":
        dpsgd = False
    else:
        dpsgd = True
    model = build_models(model_type)
    if dpsgd:
        optimizer = FixedDPKerasSGDOptimizer(
            batch_size = batch_size,
            num_parameters = num_parameters,
            l2_norm_clip=l2_norm_clip,
            noise_multiplier=noise_multiplier,
            var_list = model.trainable_variables,
            #num_microbatches=batch_size//microbatch_size,
            microbatch_size = microbatch_size,
            learning_rate=learning_rate)
        # Compute vector of per-example loss rather than its mean over a minibatch.
        loss = tf.keras.losses.CategoricalCrossentropy(
            from_logits=True, reduction=tf.losses.Reduction.NONE)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
        loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile model with Keras
    checkpoint_filepath = "./dp_data/"+file_name
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True)
    #early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    ##    monitor='val_loss', min_delta=0, patience=5, verbose=0,
    #    mode='auto', baseline=None, restore_best_weights=True
    #)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    # Train model with Keras
    if dpsgd:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=microbatch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    else:
        history = model.fit(train_data, train_labels,
                epochs=epochs,
                validation_data=(test_data, test_labels),
                batch_size=batch_size, 
                callbacks = [model_checkpoint_callback], workers=1) # , early_stopping_callback
    
    #evaluated_result = model.evaluate(
    #    x=test_data, y=test_labels, batch_size=None, verbose=1, sample_weight=None, steps=None,
    #    callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False,
    #    return_dict=True)
    if model_type not in accuracies.keys():
        accuracies[model_type] = []
    accuracies[model_type].append(history)
    #accuracies[model_type].append(evaluated_result["accuracy"])

# Compute the privacy budget expended.
#if dpsgd:
#    eps = compute_epsilon(epochs * 60000 // batch_size)
#    print('For delta=1e-5, the current epsilon is: %.2f' % eps)
#else:
#    print('Trained with vanilla non-private SGD optimizer')

In [ ]:
accuracies

In [ ]:
train_data

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

# Using set_dashes() to modify dashing of an existing line
for key in parameter_list.keys():
    line1, = ax.plot(parameter_list[key],accuracies[key],  label=key)
    line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

ax.legend()
plt.show()

In [ ]:
ee
import pickle
result = open("result","wb")
pickle.dump(accuracies, result)
pickle.dump(parameter_list, result)
result.close()


In [ ]:
import pickle dd

In [ ]:
result = open("result", "rb")
accuracies = pickle.load(result)
parameter_list = pickle.load(result)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(num)

In [ ]:
parameter_list

In [ ]:

fig, ax = plt.subplots()

# Using set_dashes() to modify dashing of an existing line
line1, = ax.plot(parameter_list["linear"],accuracies["linear"],  label='Linear')
line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

line2, = ax.plot(parameter_list["bias"],  accuracies["bias"], label='Bias')
line2.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

ax.legend()
plt.show()

In [ ]:
#Test
a = [tf.constant([2., 3, 4.], shape=(3,), dtype=tf.float32), tf.constant([3., 4, 5], shape=(3,), dtype=tf.float32), tf.constant([2., 3, 4.], shape=(3,), dtype=tf.float32)]
b = tf.ones_like(a)
b

In [ ]:
tf.linalg.global_norm(a[:2])

In [ ]:
tf.clip_by_global_norm(a, 1)[0]

In [ ]:
tf.Variable(0)

In [ ]:
tf.